In [2]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.29/01


In [3]:
isigma = 2
N_files = 1
N_centr = 5
N_hists = 1
N_layers = 3
N_ilyaers = 8
is_sigmalised = 0
iOption0 = 0
iOption1 = 0
centralities = ["0-20%","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,5,5]
layer_bins = [1,2,3,4,5,6]
colors=[1,2,4,root.kGreen+2,root.kMagenta,3]
file_path="input/"
file_names="my-fifth_QA_19081_664runs"
hist_select_3D_names = ["DCA_hist"]

In [4]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{icentr}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
            #if icentr==4: hist_select_3D0[-1].Add(infile.Get(hist_select_3D_names[ihist]+f"_{icentr-1}"))
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/my-fifth_QA_19081_664runs0.root


In [5]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.4,0.6,0.8,1.0,1.4,2.0,2.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append(0)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)

pt =  array('d', [0.30000000000000004, 0.5, 0.7, 0.9, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.4, 0.6, 0.8, 1.0, 1.4, 2.0, 2.8, 4.9])


In [6]:
boards_tot = [[-1000,1000],[-1000,1000]]
boards_fit = [[-150,300],[-150,300]]

In [7]:
def emc_fitter(InHist, ilayer, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=1
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}",layer_bins[2*ilayer],layer_bins[2*ilayer+1],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        #print("CENTRLITY :", centrality[2*centr], "-",centrality[2*centr+1])
        #print("PT :", pt_boarders[p], "-",pt_boarders[p+1])
        #print("bins :", pt_boarders[p], "-",pt_boarders[p+1])
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus{seed+p}', 'gaus',  boards_fit[int(seed0%2)][0], boards_fit[int(seed0%2)][1])
        g_pol = root.TF1( f'g_pol{seed+p}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(seed0%2)][0],boards_tot[int(seed0%2)][1])
        Format_Func(g_gaus,3,4)
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}', 'gaus(0)+[3]+[4]*x*(abs(x)/x-1)+[5]*x*(abs(x)/x+1)',boards_tot[int(seed0%2)][0], boards_tot[int(seed0%2)][1])
        Format_Func(total,2,6)


        if fits_mean!=0 and fits_sigma!=0:
            mean_ii=fits_sigma.Eval(pt[p])
            sigma_ii=fits_sigma.Eval(pt[p])
            g_gaus.SetParameter(1,sigma_ii)
            g_gaus.SetParameter(2,sigma_ii)
            g_gaus.SetParLimits(1,mean_ii*0.8-0.05,mean_ii*1.2+0.05)
            g_gaus.SetParLimits(2,sigma_ii*0.5,sigma_ii*1.5)
        else:
            g_gaus.SetParameter(1,0.)
            g_gaus.SetParameter(2,boards_fit[int(seed0%2)][1])


        #emc_var_porj.GetXaxis().SetRangeUser(-1.4, 0.5);
        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(seed0%2)][0]/10,boards_tot[int(seed0%2)][1]/10)
        total.SetParLimits(2,boards_fit[int(seed0%2)][1]/10,boards_tot[int(seed0%2)][1]/2)

        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]

        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
        Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))

        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")
        m2_name=str(icentr)+"/"+str(ilayer)+"/emc_"+str(round(pt[p],1))

        if fits_mean==0:
            c1.SaveAs( "output/DCA_fits/" + m2_name + ".png" )
        else:
            c1.SaveAs( "output/DCA_fits/" + m2_name + "upd.png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [8]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists1 = []
        for ilayer in layers:
            out_hists1.append(emc_fitter(InHist[seed][icentr][iOption0+iOption1*2], ilayer, seed, icentr, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [9]:
import multiprocess as mp

Ntr = N_files

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2], i, isigma) for i in range(Ntr)])
pool.close()

In [10]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1 = [], [], [], [], [], []
    for icentr in range(N_centr):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
        for ilayer in range(N_layers):
            Means0.append(output_array[ifile][icentr][ilayer][0])
            Mean_errs0.append(output_array[ifile][icentr][ilayer][1])
            Sigmas0.append(output_array[ifile][icentr][ilayer][2])
            Sigma_errs0.append(output_array[ifile][icentr][ilayer][3])
            Int_FG0.append(output_array[ifile][icentr][ilayer][4])
            Int_BG0.append(output_array[ifile][icentr][ilayer][5])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [20]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
colors=[1,2,4,6,root.kGreen+2]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,80)
            fit_functions1[-1].SetParameter(1,200)
            fit_functions1[-1].SetParameter(2,-2)
            fit_functions1[-1].SetParLimits(2,-3.526,-0.023)
            fit_functions1[-1].SetParameter(3,-5)
            fit_functions1[-1].SetParLimits(3,-30.526,-0.023)
            fit_functions1[-1].SetLineColor(colors[ilayer%4])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [21]:

print(Sigmas[0][0][0],Sigmas[0][3][0])
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1500,900)
    c3.Divide(3,2)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = 0, 400
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Sigmas[ifile][icentr][ilayer],pt_err,Sigma_errs[ifile][icentr][ilayer]))
            #g_sigma_inter[icentr][ilayer].append(root.TGraphErrors(N_pt))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            #Format_Graph(g_sigma_inter[icentr][ilayer][-1], 21, 0, colors[ilayer], 3, colors[ilayer], 1, 1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_functions[ifile][icentr][ilayer%4],"Q","")
            #params_mean_sigma[syst][centr][0].append(all_fit_sigma_low[ii].GetParameters())
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            #g_sigma_inter[icentr][ilayer][-1].Draw('sameL')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {ilayer}","p")
            #legends[-1].AddEntry(fit_functions[icentr][ilayer],f"fit","l")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{iOption1}.png")

array('d', [192.82310004462792, 120.0345611508149, 95.4104266011024, 85.92179924228166, 80.41611270592223, 76.41674316006058, 76.33501793443014, 78.30029531716333]) array('d', [179.77313786611984, 128.6178952177692, 103.56544753480196, 91.11508661132044, 83.24592483206278, 77.12353777823913, 75.63606605757349, 71.6495509657718])


python ERROR: cannot open image file "output/Sigmas/sigma_000.png" for writing. Please check permissions.


In [23]:
print(f"const float params[{N_files}][{1}][{N_layers}][4]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    fit_functions_icentr = fit_functions_ifile[4]
    print("{")
    for fits in fit_functions_icentr:
        print("{"+f"{round(fits.GetParameter(0),3)}, {round(fits.GetParameter(1),3)}, {round(fits.GetParameter(2),3)}"+"},")
    print("},") 
    print("},")      
print("};")        

const float params[1][1][3][4]={
{
{
{57.69, 219.761, -2.441},
{59.223, 248.853, -1.92},
{60.882, 353.445, -2.1},
},
},
};


In [14]:
print(f"params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}, {round(fits.GetParameter(3),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

params[1][5][3][3]={
{
{
{74.997648, 299.890396, -3.526, -0.023},
{82.85642, 440.167721, -3.526, -6.269993},
{95.434217, 567.809585, -3.526, -9.834476},
},
{
{76.377664, 301.447934, -3.526, -0.023},
{81.338487, 474.788306, -3.526, -5.387999},
{88.495018, 645.797707, -3.526, -7.319972},
},
{
{77.942215, 297.232835, -3.526, -0.766481},
{83.907628, 451.617415, -3.526, -6.419201},
{85.24134, 676.547116, -3.526, -6.882334},
},
{
{81.469422, 284.312851, -3.526, -2.555189},
{84.482913, 356.802038, -3.176929, -6.112976},
{86.101174, 679.632458, -3.526, -6.693458},
},
{
{74.310949, 249.541164, -3.23391, -6.72977},
{89.393515, 284.74271, -2.853937, -10.878751},
{104.957914, 490.867599, -3.526, -16.128482},
},
},
};


In [15]:
colors=[1,2,4,6,root.kGreen+2]
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.029)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-2)
            fit_functions1[-1].SetParLimits(2,-3.126,-0.023)
            #fit_functions1[-1].FixParameter(1,0.)
            #fit_functions1[-1].FixParameter(2,1)
            #fit_functions1[-1].SetParLimits(2,-3.126,-0.023)
            fit_functions1[-1].SetLineColor(colors[ilayer%4])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [16]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3",f"c3",1500,900)
    c3.Divide(3,2)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -500, 500
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Means[ifile][icentr][ilayer],pt_err,Mean_errs[ifile][icentr][ilayer]))
            #g_sigma_inter[icentr][ilayer].append(root.TGraphErrors(N_pt))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            #Format_Graph(g_sigma_inter[icentr][ilayer][-1], 21, 0, colors[ilayer], 3, colors[ilayer], 1, 1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer%4],"Q","")
            #params_mean_sigma[syst][centr][0].append(all_fit_sigma_low[ii].GetParameters())
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            #g_sigma_inter[icentr][ilayer][-1].Draw('sameL')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {ilayer}","p")
            #legends[-1].AddEntry(fit_functions[icentr][ilayer],f"fit","l")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

python ERROR: cannot open image file "output/Sigmas/mean_000.png" for writing. Please check permissions.


In [19]:
print(f"const float params[{N_files}][{1}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_mean_functions:
    print("{")
    fit_functions_icentr = fit_functions_ifile[4]
    print("{")
    for fits in fit_functions_icentr:
        print("{"+f"{round(fits.GetParameter(0),3)}, {round(fits.GetParameter(1),3)}, {round(fits.GetParameter(2),3)}"+"},")
    print("},") 
    print("},")      
print("};")        

const float params[1][1][3][3]={
{
{
{6.71, 165.566, -1.94},
{10.918, 148.468, -1.228},
{8.95, 148.93, -1.005},
},
},
};
